In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)


In [3]:
pyg_dataset = torch.load('../../data/curated dataset/cleaned_frag_pyg_dataset.pth')

C:\Users\sarab\AppData\Local\Temp\ipykernel_30688\3827785572.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load('../../data/curated dataset/cleaned

In [4]:
node_features = pyg_dataset[0].x.shape[1]  # Should be 134 based on your description
edge_features = pyg_dataset[0].edge_attr.shape[1]  # Should be 6 based on your description


In [5]:
def loss_function(recon_x, x, recon_edge_attr, edge_attr):
    node_recon_loss = F.mse_loss(recon_x, x)
    edge_recon_loss = F.mse_loss(recon_edge_attr, edge_attr)
    return node_recon_loss + edge_recon_loss

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class Encoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.conv1 = SAGEConv(node_features + edge_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, edge_index, edge_attr):
        edge_attr_expanded = torch.zeros((x.size(0), edge_features), device=x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        x = torch.cat([x, edge_attr_expanded], dim=1)
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        z = self.fc_mu(x)
        return z


class ReverseDiffusion(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(ReverseDiffusion, self).__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.node_decoder = nn.Linear(hidden_dim, node_features)
        self.edge_decoder = nn.Linear(hidden_dim * 2, edge_features)

    def forward(self, z, edge_index):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        reconstructed_x = self.node_decoder(h)

        # Compute edge features
        row, col = edge_index
        edge_h = torch.cat([h[row], h[col]], dim=1)
        reconstructed_edge_attr = self.edge_decoder(edge_h)

        return reconstructed_x, reconstructed_edge_attr

class DiffusionModel(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(DiffusionModel, self).__init__()
        self.encoder = Encoder(node_features, edge_features, hidden_dim, latent_dim)
        self.reverse_diffusion = ReverseDiffusion(latent_dim, hidden_dim, node_features, edge_features)

    def forward(self, x, edge_index, edge_attr, t):
        z = self.encode(x, edge_index, edge_attr, t)
        reconstructed_x, reconstructed_edge_attr = self.reverse_diffusion(z, edge_index)
        return reconstructed_x, reconstructed_edge_attr

    def encode(self, x, edge_index, edge_attr, t):
        z = self.encoder(x, edge_index, edge_attr)
        noise = torch.randn_like(z) * (t / 100)  # Adjust noise level based on timestep
        return z + noise

    def generate_multiple_outputs(self, x, edge_index, edge_attr, num_samples=5, num_diffusion_steps=10):
        outputs = []
        with torch.no_grad():
            for _ in range(num_samples):
                # Sample a random timestep for generating new outputs
                t = torch.randint(0, num_diffusion_steps + 1, (1,)).item()
                z_noisy = self.encode(x, edge_index, edge_attr, t)
                reconstructed_x, reconstructed_edge_attr = self.reverse_diffusion(z_noisy, edge_index)
                outputs.append((reconstructed_x, reconstructed_edge_attr, edge_index))
        return outputs



In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=60

In [8]:
import optuna
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from multiprocessing import cpu_count

# Define the objective function for hyperparameter tuning
def objective(trial):
    # Define hyperparameters to search
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)
    # Print the current trial's hyperparameters
    print(f"\n[Trial {trial.number}] Starting with parameters: {trial.params}")

    # Prepare data loaders
    train_ratio = 0.8
    valid_ratio = 0.2

    # Calculate the number of training and validation samples
    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train

    # Split the dataset into training and validation sets
    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])

    # Create DataLoader for training dataset
    train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Create DataLoader for validation dataset
    val_loader = PyGDataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model with current trial's hyperparameters
    model = DiffusionModel(node_features, edge_features, hidden_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience // 2, verbose=True)

    # Early stopping parameters
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            optimizer.zero_grad()
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            edge_attr = batch.edge_attr.to(device)

            recon_x, recon_edge_attr = model(x, edge_index, edge_attr, epoch)
            loss = loss_function(recon_x, x, recon_edge_attr, edge_attr)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_batch in val_loader:
                x_val = val_batch.x.to(device)
                edge_index_val = val_batch.edge_index.to(device)
                edge_attr_val = val_batch.edge_attr.to(device)

                recon_x_val, recon_edge_attr_val = model(x_val, edge_index_val, edge_attr_val, epoch)
                val_loss += loss_function(recon_x_val, x_val, recon_edge_attr_val, edge_attr_val).item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"[Trial {trial.number}] Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            break

    return best_val_loss


# Use multiprocessing to parallelize trials
if __name__ == "__main__":
    num_jobs = min(cpu_count(), 7)  # Use up to 7 cores (or fewer if limited)
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=num_jobs)

    # Output the best parameters along with the trial number
    best_trial = study.best_trial
    print(f"\nBest Trial Number: {best_trial.number}")
    print("Best Parameters:", best_trial.params)
    print("Best Value (Objective):", best_trial.value)


c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-12-06 19:27:39,104] A new study created in memory with name: no-name-c89329dd-130a-42d3-87a1-d387555068ac



[Trial 0] Starting with parameters: {'hidden_dim': 384, 'latent_dim': 96, 'learning_rate': 0.0005087224069211887, 'batch_size': 32, 'patience': 7}

[Trial 1] Starting with parameters: {'hidden_dim': 512, 'latent_dim': 96, 'learning_rate': 0.0019248303604977727, 'batch_size': 32, 'patience': 6}

[Trial 2] Starting with parameters: {'hidden_dim': 384, 'latent_dim': 128, 'learning_rate': 0.0009131531003322298, 'batch_size': 16, 'patience': 3}

[Trial 3] Starting with parameters: {'hidden_dim': 64, 'latent_dim': 64, 'learning_rate': 0.0014679089020835927, 'batch_size': 16, 'patience': 10}

[Trial 4] Starting with parameters: {'hidden_dim': 128, 'latent_dim': 32, 'learning_rate': 0.00643375977486633, 'batch_size': 64, 'patience': 3}

[Trial 5] Starting with parameters: {'hidden_dim': 384, 'latent_dim': 32, 'learning_rate': 1.536801280254897e-05, 'batch_size': 16, 'patience': 6}

[Trial 6] Starting with parameters: {'hidden_dim': 128, 'latent_dim': 32, 'learning_rate': 0.012247478371523481,

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[Trial 4] Epoch 1/60, Training Loss: 0.1087, Validation Loss: 0.0526
[Trial 4] Epoch 2/60, Training Loss: 0.0377, Validation Loss: 0.0232
[Trial 0] Epoch 1/60, Training Loss: 0.0603, Validation Loss: 0.0192
[Trial 1] Epoch 1/60, Training Loss: 0.0560, Validation Loss: 0.0135
[Trial 3] Epoch 1/60, Training Loss: 0.0491, Validation Loss: 0.0155
[Trial 6] Epoch 1/60, Training Loss: 0.0322, Validation Loss: 0.0140
[Trial 4] Epoch 3/60, Training Loss: 0.0184, Validation Loss: 0.0141
[Trial 5] Epoch 1/60, Training Loss: 0.1815, Validation Loss: 0.0937
[Trial 2] Epoch 1/60, Training Loss: 0.0339, Validation Loss: 0.0077
[Trial 4] Epoch 4/60, Training Loss: 0.0116, Validation Loss: 0.0091
[Trial 0] Epoch 2/60, Training Loss: 0.0135, Validation Loss: 0.0092
[Trial 1] Epoch 2/60, Training Loss: 0.0087, Validation Loss: 0.0063
[Trial 3] Epoch 2/60, Training Loss: 0.0108, Validation Loss: 0.0079
[Trial 4] Epoch 5/60, Training Loss: 0.0082, Validation Loss: 0.0069
[Trial 6] Epoch 2/60, Training Los

[I 2024-12-06 19:33:23,117] Trial 4 finished with value: 0.002953436908622583 and parameters: {'hidden_dim': 128, 'latent_dim': 32, 'learning_rate': 0.00643375977486633, 'batch_size': 64, 'patience': 3}. Best is trial 4 with value: 0.002953436908622583.


...

[Trial 197] Epoch 57/60, Training Loss: 0.0015, Validation Loss: 0.0015
[Trial 199] Epoch 48/60, Training Loss: 0.0016, Validation Loss: 0.0017
[Trial 198] Epoch 49/60, Training Loss: 0.0016, Validation Loss: 0.0018
[Trial 199] Epoch 49/60, Training Loss: 0.0016, Validation Loss: 0.0018
[Trial 198] Epoch 50/60, Training Loss: 0.0016, Validation Loss: 0.0018
[Trial 199] Epoch 50/60, Training Loss: 0.0017, Validation Loss: 0.0018
[Trial 198] Epoch 51/60, Training Loss: 0.0016, Validation Loss: 0.0018
[Trial 199] Epoch 51/60, Training Loss: 0.0016, Validation Loss: 0.0017
[Trial 198] Epoch 52/60, Training Loss: 0.0016, Validation Loss: 0.0018
[Trial 199] Epoch 52/60, Training Loss: 0.0017, Validation Loss: 0.0018
[Trial 198] Epoch 53/60, Training Loss: 0.0015, Validation Loss: 0.0017
[Trial 199] Epoch 53/60, Training Loss: 0.0016, Validation Loss: 0.0018
[Trial 198] Epoch 54/60, Training Loss: 0.0015, Validation Loss: 0.0017
[Trial 199] Epoch 54/60, Training Loss: 0.0015, Validation Loss:

[I 2024-12-07 10:48:07,858] Trial 198 finished with value: 0.0017207668349631907 and parameters: {'hidden_dim': 448, 'latent_dim': 128, 'learning_rate': 0.001076460442165689, 'batch_size': 8, 'patience': 9}. Best is trial 130 with value: 0.0014705338303125852.


[Trial 198] Epoch 60/60, Training Loss: 0.0015, Validation Loss: 0.0017


[I 2024-12-07 10:48:11,783] Trial 199 finished with value: 0.0016874009087284017 and parameters: {'hidden_dim': 448, 'latent_dim': 128, 'learning_rate': 0.0010012469632944363, 'batch_size': 8, 'patience': 9}. Best is trial 130 with value: 0.0014705338303125852.


[Trial 199] Epoch 60/60, Training Loss: 0.0015, Validation Loss: 0.0017

Best Trial Number: 130
Best Parameters: {'hidden_dim': 448, 'latent_dim': 128, 'learning_rate': 0.0008631209327259007, 'batch_size': 8, 'patience': 8}
Best Value (Objective): 0.0014705338303125852
